# Get ROIs from raw probability maps

Read the raw probability maps saved by DeXtrusion and calculate and save the ROIs positions from it.
The raw probability maps should have been saved in the `results` folder and called `FileName_eventname_rawproba.tif` if the `saved_raw_proba` option was selected when running DeXtrusion.

If the raw probability were not saved but the probability maps were saved, it is still possible to use these maps to get the ROIs, by renaming them "*_rawproba.tif" instead of "*_proba.tif".
However, the size of the probability maps should be the rescaled image shape to fit with the correct volume threshold.
If there was some rescaling of the image in the detection, either resize your probamap to the shape that had been printed during DeXtrusion run, or adjust the volume threshold.

In [17]:
talkative = True  ## print info messages
import os
from glob import glob
import numpy as np
from dextrusion.DeXtrusion import DeXtrusion
dexter = DeXtrusion(verbose=talkative)

Tensorflow with Cuda: False
Tensorflow version: 2.15.0
Num GPUs Available:  0


In [15]:
catnames = ["", "_cell_death.zip", "_cell_sop.zip", "_cell_division.zip"]   # name of events the network was trained with
dexter.catnames = catnames

test_path = "/Users/yuyangsmacbook/project/data/notum_retrain_mix/test"     # path of the images to get ROIs from
res = test_path+"results/"         # output folder where the probamaps have been saved

cat = 1                            # type of event to analyse

volume_thres = 800                 # threshold of event volume to keep it (to decrease false positives)
prob_thres = 180                   # threshold of probability of the volume to keep it (decrease false positives)

resfiles = glob(test_path+"*.tif", recursive = False)
for resfile in resfiles:
    testname = os.path.basename(resfile)
    ind = testname.find(".tif")
    testname = testname[0:ind]
    print(testname)
    outfile = res+testname+catnames[cat]
    dexter.get_rois_fromrawproba_path( imagepath=test_path+testname+".tif", cat=cat, 
                                  volume_threshold=volume_thres, proba_threshold=prob_thres, disxy=10, dist=4, outfile=outfile )

The following part used to derive the Fiji ROI through probability maps.

In [1]:
import os, glob, shutil, numpy as np
import tifffile as tiff
from dextrusion.DeXtrusion import DeXtrusion

# -----------------
# Paths
# -----------------
test_path = "/Users/yuyangsmacbook/project/data/notum_retrain_mix/test"         # Only used for basename style (GT not required here)
res       = "/Users/yuyangsmacbook/project/data/notum_retrain_mix/test/results" # Folder containing proba maps & where ROI zips will be written
os.makedirs(res, exist_ok=True)

# -----------------
# Category
# -----------------
cat = 1
catnames = ["", "_cell_death.zip", "_cell_sop.zip", "_cell_division.zip"]
suf_zip   = catnames[cat]                  # "_cell_death.zip"
suf_nozip = suf_zip.replace(".zip","")     # "_cell_death"

# -----------------
# Thresholds
# -----------------
volume_thres      = 800
prob_thres_uint   = 180    # for 8/16-bit integer proba maps (0..255/65535)
prob_thres_float  = 0.5    # for float proba maps in [0,1]
distxy_merge, distt_merge = 10, 4

# -----------------
# (1) Normalize filenames to *_rawproba.tif as expected by the library
#     Accept both "*_cell_death_proba.tif" and "*_cell_death.zip_proba.tif"
# -----------------
pat_zip   = "*" + suf_zip   + "_proba.tif"    # *_cell_death.zip_proba.tif
pat_nozip = "*" + suf_nozip + "_proba.tif"    # *_cell_death_proba.tif
probas = glob.glob(os.path.join(res, pat_nozip)) or glob.glob(os.path.join(res, pat_zip))

to_use = []
for p in probas:
    name = os.path.basename(p).replace("_proba.tif","")   # e.g. 000_029_cell_death[.zip]
    stem = name.replace(".zip","")                        # e.g. 000_029_cell_death
    if not stem.endswith(suf_nozip):
        print("Skip (suffix mismatch):", p)
        continue
    base = stem[:-len(suf_nozip)]                         # e.g. 000_029
    rawp = os.path.join(res, base + suf_nozip + "_rawproba.tif")
    if os.path.abspath(p) != os.path.abspath(rawp):
        shutil.copy2(p, rawp)   # copy to the filename expected by the library
        print("⇒ Copied as:", os.path.basename(rawp))
    to_use.append((base, rawp))

print(f"Usable proba maps: {len(to_use)}")

# -----------------
# (2) Call DeXtrusion: it looks for *_rawproba.tif under outpath and writes ROI zips
# -----------------
dexter = DeXtrusion(verbose=True)
dexter.catnames = catnames
dexter.outpath  = res   # IMPORTANT: get_rois_fromrawproba_path will look under outpath for *_rawproba.tif

def proba_to_roi_from_base(base):
    # choose threshold scale according to dtype
    rawp = os.path.join(res, base + suf_nozip + "_rawproba.tif")
    img = tiff.imread(rawp)
    pthr = float(prob_thres_float) if np.issubdtype(img.dtype, np.floating) else int(prob_thres_uint)

    # imagepath is only used to provide the basename; the file itself is not read
    fake_imagepath = os.path.join(test_path, base + ".tif")
    outzip = os.path.join(res, base + suf_zip)

    dexter.get_rois_fromrawproba_path(
        imagepath=fake_imagepath,     # only the basename "base" is used
        cat=cat,
        volume_threshold=volume_thres,
        proba_threshold=pthr,
        disxy=distxy_merge,
        dist=distt_merge,
        outfile=outzip
    )
    return outzip

wrote = 0
for base, rawp in to_use:
    try:
        outzip = proba_to_roi_from_base(base)
        wrote += 1
        print("✅ ROI written:", os.path.basename(outzip))
    except Exception as e:
        print("❌ Failed:", base, "→", e)

print(f"[Done] {wrote} ROI zip(s) written to {res}")


⇒ Copied as: 200_229_cell_death_rawproba.tif
⇒ Copied as: 000_029_cell_death_rawproba.tif
Usable proba maps: 2
Tensorflow with Cuda: False
Tensorflow version: 2.15.0
Num GPUs Available:  0
Writing 5 Rois in file /Users/yuyangsmacbook/project/data/notum_retrain_mix/test/results/200_229_cell_death.zip
✅ ROI written: 200_229_cell_death.zip
Writing 6 Rois in file /Users/yuyangsmacbook/project/data/notum_retrain_mix/test/results/000_029_cell_death.zip
✅ ROI written: 000_029_cell_death.zip
[Done] 2 ROI zip(s) written to /Users/yuyangsmacbook/project/data/notum_retrain_mix/test/results
